# The Scalar Wave Equation: Creating an Einstein Toolkit Initial Data thorn

### NRPy+ Source Code for this module: [ScalarWave/InitialData_PlaneWave.py](../edit/ScalarWave/InitialData_PlaneWave.py) , which is fully documented in the [previous NRPy+ tutorial module on using NRPy+ to construct these scalar wave initial data as SymPy expressions](Tutorial-ScalarWave.ipynb).


## Scalar Wave Initial Data

Now, we will move on to the second thorn. This thorn will set the initial data.

### Step 1: Call on NRPy+ to convert the SymPy expressions for the Weyl scalars and associated invariants into one C-code kernel for each. 

The plane wave initial data are already constructed by [ScalarWave/InitialData_PlaneWave.py](../edit/ScalarWave/InitialData_PlaneWave.py), so we can simply import them to use here.

We will now output the C code for the header file to set plane wave initial data that will be evolved by the first thorn we made in this tutorial.

In [1]:
# Step 1a: Import needed NRPy+ core modules:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop

# Step 1b: This is an Einstein Toolkit (ETK) thorn. Here we
#          tell NRPy+ that gridfunction memory access will 
#          therefore be in the "ETK" style.
par.set_parval_from_str("grid::GridFuncMemAccess","ETK")

# Step 1c: Call the InitialData_PlaneWave() function from within the
#         ScalarWave/InitialData_PlaneWave.py module,
#         which should do exactly the same as in Steps 1-5 above.
import ScalarWave.InitialData_PlaneWave as swid

# Step 2: Within the ETK, the 3D gridfunctions x, y, and z store the
#         Cartesian grid coordinates. Setting the gri.xx[] arrays
#         to point to these gridfunctions forces NRPy+ to treat
#         the Cartesian coordinate gridfunctions properly --
#         reading them from memory as needed.
x,y,z = gri.register_gridfunctions("AUX",["x","y","z"])
gri.xx[0] = x
gri.xx[1] = y
gri.xx[2] = z

# Step 3: Set up the plane wave initial data. This sets uu_ID and vv_ID.
swid.InitialData_PlaneWave()

# Step 4: Register uu and vv gridfunctions so they can be written to by NRPy.
uu,vv = gri.register_gridfunctions("EVOL",["uu","vv"])

# Step 5: Set the uu and vv gridfunctions to the uu_ID & vv_ID variables 
#         defined by InitialData_PlaneWave().
uu = swid.uu_ID
vv = swid.vv_ID

# Step 6: Create directories for the thorn if they don't exist.
!mkdir IDScalarWaveNRPy     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir IDScalarWaveNRPy/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.

# Step 7: Create the C code output kernel.
scalar_PWID_to_print = [\
                        lhrh(lhs=gri.gfaccess("out_gfs","uu"),rhs=uu),\
                        lhrh(lhs=gri.gfaccess("out_gfs","vv"),rhs=vv),]
scalar_PWID_CcodeKernel = fin.FD_outputC("returnstring",scalar_PWID_to_print)

scalar_PWID_looped = loop.loop(["i2","i1","i0"],["1","1","1"],["cctk_lsh[2]-1","cctk_lsh[1]-1","cctk_lsh[0]-1"],\
                               ["1","1","1"],["#pragma omp parallel for","",""],"",\
                               scalar_PWID_CcodeKernel.replace("time","cctk_time"))

# Step 8: Write the C code kernel to file.
with open("IDScalarWaveNRPy/src/ScalarWave_PWID.h", "w") as file:
    file.write(str(scalar_PWID_looped))


### Step 2: Interfacing with the Einstein Toolkit

#### Step 2a: Constructing the Einstein Toolkit calling functions that include the C code kernels.

We will write another C file with the functions we need here.

In [2]:
%%writefile IDScalarWaveNRPy/src/InitialData.c
#include <math.h>

#include "cctk.h"
#include "cctk_Parameters.h"
#include "cctk_Arguments.h"

void IDScalarWaveNRPy_InitialData(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS
  DECLARE_CCTK_PARAMETERS
  
  const CCTK_REAL *xGF = x;
  const CCTK_REAL *yGF = y;
  const CCTK_REAL *zGF = z;
#include "ScalarWave_PWID.h"
}

Overwriting IDScalarWaveNRPy/src/InitialData.c


### Step 2b: CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure

Writing a module ("thorn") within the Einstein Toolkit requires that three "ccl" files be constructed, all in the root directory of the thorn:

1. $\text{interface.ccl}$: defines the gridfunction groups needed, and provides keywords denoting what this thorn provides and what it should inherit from other thorns.
1. $\text{param.ccl}$: specifies free parameters within the thorn.
1. $\text{schedule.ccl}$: allocates storage for gridfunctions, defines how the thorn's functions should be scheduled in a broader simulation, and specifies the regions of memory written to or read from gridfunctions.

We will also need to write an interface file for this thorn.

In [3]:
%%writefile IDScalarWaveNRPy/interface.ccl
implements: IDScalarWaveNRPy
inherits: WaveToyNRPy grid


Overwriting IDScalarWaveNRPy/interface.ccl


We will need to write a param file for this thorn.

In [4]:
%%writefile IDScalarWaveNRPy/param.ccl
shares: grid

USES KEYWORD type

shares: WaveToyNRPy
    
USES REAL wavespeed

restricted:
CCTK_KEYWORD initial_data "Type of initial data"
{
  "plane"      :: "Plane wave"
} "plane"

restricted:
CCTK_REAL kk0 "The wave number in the x-direction"
{
 *:* :: "No restriction"
} 4.0

restricted:
CCTK_REAL kk1 "The wave number in the y-direction"
{
 *:* :: "No restriction"
} 0.0

restricted:
CCTK_REAL kk2 "The wave number in the z-direction"
{
 *:* :: "No restriction"
} 0.0


Overwriting IDScalarWaveNRPy/param.ccl


We will need to write a schedule file for this thorn.

In [5]:
%%writefile IDScalarWaveNRPy/schedule.ccl
schedule IDScalarWaveNRPy_InitialData at CCTK_INITIAL as WaveToy_InitialData
{
  STORAGE:       WaveToyNRPy::scalar_fields[3]
  LANG:          C
  READS: grid::x(Everywhere)
  READS: grid::y(Everywhere)
  READS: grid::y(Everywhere)
  WRITES: uuGF(Everywhere)
  WRITES: vvGF(Everywhere)
} "Initial data for 3D wave equation"


Overwriting IDScalarWaveNRPy/schedule.ccl


#### Step 2c: Tell the Einstein Toolkit to compile the C code.

We will also need $\text{make.code.defn}$, which states which files need to be compiled. This thorn only has the one C file to compile so far.

In [6]:
%%writefile IDScalarWaveNRPy/src/make.code.defn
SRCS = InitialData.c

Overwriting IDScalarWaveNRPy/src/make.code.defn
